In [ ]:
flex_author = "MBET"
flex_title = "Upload PVD and Vibro Log data"


In [ ]:
from io import BytesIO
from datetime import datetime
import os

from pt_etl_tooling.pvd_cho_ghb import parse_pvd_log_data as parse_ghb_cho
from pt_etl_tooling.pvd_cof import parse_pvd_data as parse_cofra
from pt_etl_tooling.vibro import parse_vibro_sheet as parse_keller

import requests
from ipywidgets import widgets
from ipywidgets import GridspecLayout
from openpyxl import load_workbook
import sentry_sdk
from sentry_sdk import capture_exception, capture_message
from dotenv import load_dotenv
load_dotenv()

sentry_sdk.init(
    os.getenv("SENTRY_DSN"),
    traces_sample_rate=1.0
)

## Upload


### PVD

In [ ]:
pvd_endpoint = "http://10.64.32.62:8080/api/pvd/pvd"
pvd_endpoint_force = "http://10.64.32.62:8080/api/pvd/pvd-force"

def process_pvd_data(b):
    with output:
        print(f">>> Started processing PVD data on {datetime.now().strftime('%Y-%b-%d %H:%M:%S')}")
    funcs = [parse_cofra, parse_ghb_cho]
    total_files = sum([len(files) for files in [cofra_files.value, cho_ghb_files.value]])
    progress_step = 100 / total_files
    
    for i, files in enumerate([cofra_files, cho_ghb_files]):
        if len(files.value)>0:
            for name, file_info in files.value.items():
                
                wb = load_workbook(filename=BytesIO(file_info['content']))
                ws = wb.active
                
                try:
                    log_data, force_data = funcs[i](ws)
                    message = f">>> {datetime.now().strftime('%Y-%b-%d %H:%M:%S')} - Processed {name}"
                    pvd_response = requests.post(pvd_endpoint, json=log_data)
                    force_response =  requests.post(pvd_endpoint_force, json=force_data)
                except Exception as e:
                    message = f">>> {datetime.now().strftime('%Y-%b-%d %H:%M:%S')} - FAILED {name} - {e}"
                    capture_exception(e)

                progress.value = progress.value + progress_step

                capture_message(message)
        
                with output:
                    print(message)

In [ ]:
grid_pvd = GridspecLayout(2, 2)

grid_pvd[0,0] = widgets.Label("Cofra")
cofra_files = widgets.FileUpload(accept=".xlsx", multiple=True)
grid_pvd[0,1] = cofra_files

grid_pvd[1,0]  = widgets.Label("Chosuk & Geoharbour")
cho_ghb_files = widgets.FileUpload(accept=".xlsx", multiple=True)
grid_pvd[1,1] = cho_ghb_files

process = widgets.Button(description="Process PVD data")
process.on_click(process_pvd_data)

progress = widgets.FloatProgress(value=0, min=0, max=100)

vbox_pvd = widgets.VBox([grid_pvd, process, progress])

vbox_pvd

### Vibro

In [ ]:
vibro_endpoint = "http://10.64.32.62:8080/api/vibro/vibro"

def process_vibro_data(b):
    total_files = len(keller_files.value)
    progress_step = 100 / total_files
    with output:
        print(f">>> Started processing PVD data on {datetime.now().strftime('%Y-%b-%d %H:%M:%S')}")
                       
    if total_files > 0:
        for name, file_info in keller_files.value.items():
            wb = load_workbook(filename=BytesIO(file_info['content']))
            ws = wb.active
            
            try:
                log_data, force_data = parse_keller(ws)
                vibro_response = requests.post(vibro_endpoint, json=log_data)
                message = f">>> {datetime.now().strftime('%Y-%b-%d %H:%M:%S')} - Processed {name}"
            except Exception as e:
                message = f">>> {datetime.now().strftime('%Y-%b-%d %H:%M:%S')} - FAILED {name} - {e}"
                capture_exception(e)

            progress_vibro.value = progress_vibro.value + progress_step

            capture_message(message)

            with output:
                print(message)

In [ ]:
grid_vibro = GridspecLayout(1, 2)

keller_files = widgets.FileUpload(accept=".xlsx", multiple=True)
grid_vibro[0,0] = widgets.Label("Keller")
grid_vibro[0,1] = keller_files

process_vibro = widgets.Button(description="Process Vibro data")
process_vibro.on_click(process_vibro_data)
progress_vibro = widgets.FloatProgress()

vbox_vibro = widgets.VBox([grid_vibro, process_vibro, progress_vibro])

vbox_vibro


## Log

### log

In [ ]:
output = widgets.Output()
output